In [ ]:
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import os

class AnswerSheetGenerator:
    def __init__(self, num_questions=50, choices_per_question=4, sheet_width=800, sheet_height=1000):
        self.num_questions = num_questions
        self.choices_per_question = choices_per_question
        self.sheet_width = sheet_width
        self.sheet_height = sheet_height
        self.circle_radius = 12
        self.question_spacing = 40
        self.choice_spacing = 60
        
    def generate_answer_sheet(self, filename="answer_sheet.png"):
        """Generate a blank answer sheet"""
        # Create white background
        img = Image.new('RGB', (self.sheet_width, self.sheet_height), 'white')
        draw = ImageDraw.Draw(img)
        
        # Try to load a font, fallback to default if not available
        try:
            font_title = ImageFont.truetype("arial.ttf", 24)
            font_text = ImageFont.truetype("arial.ttf", 12)
        except:
            font_title = ImageFont.load_default()
            font_text = ImageFont.load_default()
        
        # Draw title
        title = "MULTIPLE CHOICE ANSWER SHEET"
        draw.text((50, 30), title, fill='black', font=font_title)
        
        # Draw instructions
        instructions = "Fill in the circle completely for your answer choice"
        draw.text((50, 70), instructions, fill='black', font=font_text)
        
        # Draw answer bubbles
        start_x = 80
        start_y = 120
        
        for q in range(self.num_questions):
            # Question number
            question_text = f"Q{q+1:2d}:"
            draw.text((start_x - 50, start_y + q * self.question_spacing), 
                     question_text, fill='black', font=font_text)
            
            # Answer choices (A, B, C, D, etc.)
            for choice in range(self.choices_per_question):
                x = start_x + choice * self.choice_spacing
                y = start_y + q * self.question_spacing
                
                # Draw circle
                draw.ellipse([x - self.circle_radius, y - self.circle_radius,
                            x + self.circle_radius, y + self.circle_radius],
                           outline='black', width=2)
                
                # Draw choice letter
                choice_letter = chr(ord('A') + choice)
                draw.text((x - 5, y - 20), choice_letter, fill='black', font=font_text)
        
        # Add student info section
        info_y = start_y + (self.num_questions + 2) * self.question_spacing
        draw.text((50, info_y), "Name: ________________________", fill='black', font=font_text)
        draw.text((50, info_y + 30), "Student ID: ___________________", fill='black', font=font_text)
        draw.text((50, info_y + 60), "Date: ________________________", fill='black', font=font_text)
        
        img.save(filename)
        print(f"Blank answer sheet saved as {filename}")
        return img
    
    def generate_filled_sheet(self, answers, filename="filled_answer_sheet.png"):
        """Generate a filled answer sheet with given answers"""
        # First generate blank sheet
        img = self.generate_answer_sheet("temp_blank.png")
        
        # Convert to OpenCV format for filling circles
        img_cv = cv2.imread("temp_blank.png")
        
        start_x = 80
        start_y = 120
        
        for q, answer in enumerate(answers):
            if q >= self.num_questions:
                break
                
            if answer < self.choices_per_question:
                # Calculate position of the answer circle
                x = start_x + answer * self.choice_spacing
                y = start_y + q * self.question_spacing
                
                # Fill the circle
                cv2.circle(img_cv, (x, y), self.circle_radius - 2, (0, 0, 0), -1)
        
        cv2.imwrite(filename, img_cv)
        
        # Clean up temp file
        if os.path.exists("temp_blank.png"):
            os.remove("temp_blank.png")
        
        print(f"Filled answer sheet saved as {filename}")
        return img_cv


class AnswerSheetDetector:
    def __init__(self, num_questions=50, choices_per_question=4):
        self.num_questions = num_questions
        self.choices_per_question = choices_per_question
        
    def detect_answers(self, image_path):
        """Detect filled circles in an answer sheet"""
        # Read image
        img = cv2.imread(image_path)
        if img is None:
            raise ValueError(f"Could not load image: {image_path}")
        
        # Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Apply Gaussian blur to reduce noise
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        
        # Use HoughCircles to detect circles
        circles = cv2.HoughCircles(
            blurred,
            cv2.HOUGH_GRADIENT,
            dp=1,
            minDist=30,
            param1=50,
            param2=30,
            minRadius=8,
            maxRadius=20
        )
        
        detected_answers = []
        
        if circles is not None:
            circles = np.round(circles[0, :]).astype("int")
            
            # Calculate expected positions
            start_x = 80
            start_y = 120
            question_spacing = 40
            choice_spacing = 60
            
            for q in range(self.num_questions):
                question_answers = []
                
                for choice in range(self.choices_per_question):
                    expected_x = start_x + choice * choice_spacing
                    expected_y = start_y + q * question_spacing
                    
                    # Check if any detected circle is near this position
                    for (x, y, r) in circles:
                        distance = np.sqrt((x - expected_x)**2 + (y - expected_y)**2)
                        if distance < 25:  # Tolerance for circle detection
                            # Check if circle is filled (darker center)
                            center_intensity = gray[y, x]
                            if center_intensity < 100:  # Threshold for filled circles
                                question_answers.append(choice)
                                break
                
                if len(question_answers) == 1:
                    detected_answers.append(question_answers[0])
                elif len(question_answers) == 0:
                    detected_answers.append(-1)  # No answer
                else:
                    detected_answers.append(-2)  # Multiple answers (invalid)
        
        return detected_answers
    
    def compare_answers(self, student_answers, correct_answers):
        """Compare student answers with correct answers"""
        if len(student_answers) != len(correct_answers):
            min_len = min(len(student_answers), len(correct_answers))
            student_answers = student_answers[:min_len]
            correct_answers = correct_answers[:min_len]
        
        results = {
            'total_questions': len(correct_answers),
            'correct': 0,
            'incorrect': 0,
            'unanswered': 0,
            'invalid': 0,
            'score_percentage': 0,
            'detailed_results': []
        }
        
        for i, (student, correct) in enumerate(zip(student_answers, correct_answers)):
            question_result = {
                'question': i + 1,
                'student_answer': student,
                'correct_answer': correct,
                'status': ''
            }
            
            if student == -1:
                results['unanswered'] += 1
                question_result['status'] = 'unanswered'
            elif student == -2:
                results['invalid'] += 1
                question_result['status'] = 'invalid'
            elif student == correct:
                results['correct'] += 1
                question_result['status'] = 'correct'
            else:
                results['incorrect'] += 1
                question_result['status'] = 'incorrect'
            
            results['detailed_results'].append(question_result)
        
        # Calculate score percentage (only count answered questions)
        answered_questions = results['total_questions'] - results['unanswered']
        if answered_questions > 0:
            results['score_percentage'] = (results['correct'] / answered_questions) * 100
        
        return results
    
    def print_results(self, results):
        """Print detailed results"""
        print("\n" + "="*50)
        print("ANSWER SHEET ANALYSIS RESULTS")
        print("="*50)
        print(f"Total Questions: {results['total_questions']}")
        print(f"Correct Answers: {results['correct']}")
        print(f"Incorrect Answers: {results['incorrect']}")
        print(f"Unanswered: {results['unanswered']}")
        print(f"Invalid (Multiple marks): {results['invalid']}")
        print(f"Score: {results['score_percentage']:.1f}%")
        print("\nDetailed Results:")
        print("-" * 40)
        
        for result in results['detailed_results']:
            q_num = result['question']
            student_ans = result['student_answer']
            correct_ans = result['correct_answer']
            status = result['status']
            
            # Convert answer numbers to letters
            student_letter = chr(ord('A') + student_ans) if student_ans >= 0 else 'N/A'
            correct_letter = chr(ord('A') + correct_ans) if correct_ans >= 0 else 'N/A'
            
            if status == 'unanswered':
                student_letter = 'No Answer'
            elif status == 'invalid':
                student_letter = 'Multiple'
            
            print(f"Q{q_num:2d}: Student={student_letter:8s} Correct={correct_letter} [{status.upper()}]")


# Example usage and demonstration
def main():
    # Initialize generator and detector
    generator = AnswerSheetGenerator(num_questions=45, choices_per_question=4)
    detector = AnswerSheetDetector(num_questions=45, choices_per_question=4)
    
    print("Generating answer sheets...")
    
    # Generate blank answer sheet
    generator.generate_answer_sheet("blank_answer_sheet.png")
    
    # Generate a sample filled answer sheet
    sample_answers = [0, 1, 2, 3, 0, 1, 2, 3, 1, 2, 0, 3, 1, 0, 2, 3, 0, 1, 2, 3]  # A, B, C, D pattern
    generator.generate_filled_sheet(sample_answers, "sample_filled_sheet.png")
    
    # Detect answers from the filled sheet
    print("\nDetecting answers from filled sheet...")
    try:
        detected_answers = detector.detect_answers("sample_filled_sheet.png")
        print(f"Detected answers: {detected_answers}")
        
        # Compare with correct answers (for demonstration)
        correct_answers = [0, 1, 2, 3, 0, 1, 2, 3, 1, 2, 0, 3, 1, 0, 2, 3, 0, 1, 2, 3]
        
        results = detector.compare_answers(detected_answers, correct_answers)
        detector.print_results(results)
        
    except Exception as e:
        print(f"Error in detection: {e}")
        print("Make sure OpenCV is installed: pip install opencv-python")

if __name__ == "__main__":
    main()

Generating answer sheets...
Blank answer sheet saved as blank_answer_sheet.png
Blank answer sheet saved as temp_blank.png
Filled answer sheet saved as sample_filled_sheet.png

Detecting answers from filled sheet...
Detected answers: [0, 1, 2, 3, 0, 1, 2, 3, 1, 2, 0, 3, 1, 0, 2, 3, 0, 1, 2, 3]

ANSWER SHEET ANALYSIS RESULTS
Total Questions: 20
Correct Answers: 20
Incorrect Answers: 0
Unanswered: 0
Invalid (Multiple marks): 0
Score: 100.0%

Detailed Results:
----------------------------------------
Q 1: Student=A        Correct=A [CORRECT]
Q 2: Student=B        Correct=B [CORRECT]
Q 3: Student=C        Correct=C [CORRECT]
Q 4: Student=D        Correct=D [CORRECT]
Q 5: Student=A        Correct=A [CORRECT]
Q 6: Student=B        Correct=B [CORRECT]
Q 7: Student=C        Correct=C [CORRECT]
Q 8: Student=D        Correct=D [CORRECT]
Q 9: Student=B        Correct=B [CORRECT]
Q10: Student=C        Correct=C [CORRECT]
Q11: Student=A        Correct=A [CORRECT]
Q12: Student=D        Correct=D [COR